In [1]:
# %cd /home/ubu-admin/Developer/tensorflow-wavelets
%cd /workspaces/OpenDVCW
import os
import load
import OpenDVCW
import tensorflow as tf
from math import log10, sqrt
import cv2
import numpy as np

/workspaces/OpenDVCW


In [2]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [3]:
def compare(original_path, compressed_path, binary_path):
    original = cv2.imread(original_path)
    compressed = cv2.imread(compressed_path)
    bin_size = os.path.getsize(binary_path)
    value = PSNR(original, compressed)
    print("bin size: ", bin_size , "psnr: ", value)

In [4]:
BATCH_SIZE = 1
Height = 240
Width = 240
lmbda = 256
print("Loading model")
model = tf.keras.models.load_model('model_save_checkpoints_wavelets_sym3_Lmbd_4096_es_600_I_QP_27_240x240_CosineDecay_20220421-201124')
# Show the model architecture
model.summary()

Loading model


2022-04-23 21:06:34.843103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 21:06:34.895580: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 21:06:34.896594: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "open_dvc"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mv_analysis (Sequential)    (None, None, None, 128)   659456    
                                                                 
 mv_synthesis (Sequential)   (None, None, None, 2)     642824    
                                                                 
 res_analysis (Sequential)   (None, None, None, 128)   1552640   
                                                                 
 res_synthesis (Sequential)  (None, None, None, 3)     1536015   
                                                                 
 wavelets_optical_flow (Wave  multiple                 240050    
 letsOpticalFlow)                                                
                                                                 
 motion_compensation (Motion  multiple                 486467    
 Compensation)                                            

2022-04-23 21:06:34.901718: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 21:06:34.906855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 21:06:34.907756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 21:06:34.908593: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [5]:
# !rm -r /workspaces/OpenDVCW/Test_com
# !mkdir /workspaces/OpenDVCW/Test_com
# !mkdir /workspaces/OpenDVCW/Test_com/h264
# !mkdir /workspaces/OpenDVCW/Test_com/h265
# !mkdir /workspaces/OpenDVCW/Test_com/dvcw

In [6]:
path = load.load_path_n("folder_cloud_test.npy", 0)
p_frame_out_bin = "Test_com/dvcw/p_frame_dvcw.bin"
out_decom = "Test_com/dvcw/frame1.png"
i_on_test = "Test_com/frame0.png"
p_on_test = "Test_com/frame1.png"

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)
print(p_frame)


/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00049/0289/im1.png
/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00049/0289/im2.png


In [7]:


# write inputs to disk
OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))


OpenDVCW.compress(model, i_frame, p_frame, p_frame_out_bin, 240, 240)
OpenDVCW.decompress(model, i_frame, p_frame_out_bin, out_decom, 240, 240)


original = cv2.imread(p_on_test)
compressed = cv2.imread(out_decom)
bin_size = os.path.getsize(p_frame_out_bin)
value = PSNR(original, compressed)
print("bin size: ", bin_size , "psnr: ", value)

compress


2022-04-23 21:07:34.235024: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


decompress
bin size:  2124 psnr:  41.35339316943596


In [8]:
# !rm -r /workspaces/OpenDVCW/Test_com/h265/*
# !ffmpeg -i /workspaces/OpenDVCW/Test_com/frame%d.png -c:v libx265 -b:v 1k /workspaces/OpenDVCW/Test_com/h265/out.h265
# !ffmpeg -i /workspaces/OpenDVCW/Test_com/h265/out.h265 /workspaces/OpenDVCW/Test_com/h265/h265_decom%d.png


In [9]:
# !rm -r /workspaces/OpenDVCW/Test_com/h264/*
# !ffmpeg -i /workspaces/OpenDVCW/Test_com/frame%d.png -c:v libx264 -b:v 200k /workspaces/OpenDVCW/Test_com/h264/out.h264
# !ffmpeg -i /workspaces/OpenDVCW/Test_com/h264/out.h264 /workspaces/OpenDVCW/Test_com/h264/h264_decom%d.png

In [10]:
# !ffprobe -bsfs /workspaces/OpenDVCW/Test_com/h264/out.h264

In [17]:
# h264 test
p_frame_h264_decoded = "/workspaces/OpenDVCW/Test_com/h264/2.4k/decoded_h264_2.png"
h264_bin_path = "/workspaces/OpenDVCW/Test_com/h264/2.4k/frame1.h264"
compare(p_on_test, p_frame_h264_decoded, h264_bin_path)

bin size:  2386 psnr:  41.034551519732844


In [19]:
# h264 test
p_frame_h264_decoded = "/workspaces/OpenDVCW/Test_com/h264/3.4k/decoded_h264_2.png"
h264_bin_path = "/workspaces/OpenDVCW/Test_com/h264/3.4k/frame1.h264"
compare(p_on_test, p_frame_h264_decoded, h264_bin_path)

bin size:  3411 psnr:  41.413323638106306


In [18]:
# h264 test
p_frame_h264_decoded = "/workspaces/OpenDVCW/Test_com/h264/4.4k/decoded_h264_2.png"
h264_bin_path = "/workspaces/OpenDVCW/Test_com/h264/4.4k/frame1.h264"
compare(p_on_test, p_frame_h264_decoded, h264_bin_path)

bin size:  4364 psnr:  41.6591254443001


In [14]:
# h264 test
p_frame_h264_decoded = "/workspaces/OpenDVCW/Test_com/h264/2.6k/decoded_h264_2.png"
h264_bin_path = "/workspaces/OpenDVCW/Test_com/h264/2.6k/frame1.h264"
compare(p_on_test, p_frame_h264_decoded, h264_bin_path)

bin size:  2592 psnr:  41.60022022188281


In [21]:
# h265 test
p_frame_h265_decoded = "/workspaces/OpenDVCW/Test_com/h265/2.1k/decoded_h265_2.png"
h265_bin_path = "/workspaces/OpenDVCW/Test_com/h265/2.1k/frame1.h265"
compare(p_on_test, p_frame_h265_decoded, h265_bin_path)

bin size:  2189 psnr:  41.43568704557942


In [20]:
# h265 test
p_frame_h265_decoded = "/workspaces/OpenDVCW/Test_com/h265/2.5k/decoded_h265_2.png"
h265_bin_path = "/workspaces/OpenDVCW/Test_com/h265/2.5k/frame1.h265"
compare(p_on_test, p_frame_h265_decoded, h265_bin_path)

bin size:  2369 psnr:  41.56041669500935


bin size:  785 psnr:  39.70775637987411
